In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

tf.logging.set_verbosity(tf.logging.INFO)

In [37]:
class LeNet(object):
    def __init__(self):

        # Training data
        self.mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

        # Image params
        self.n_channels = 1
        self.image_width = 28
        self.image_height = 28

        # Training params
        self.n_classes = 10
        self.n_iter = 10000
        self.batch_size = 50
        self.activation = tf.nn.relu

        # Kernel/filter and pooling params
        self.kernel_conv = [5, 5]
        self.kernel_pool = [2, 2]
        self.padding = 2
        self.stride = 1
        self.n_filters_1 = 6
        self.n_filters_2 = 16
        self.stride_pool = 2

        # Dense params
        self.n_dense_1 = 120
        self.n_dense_2 = 84

        # Data placeholder for training or test batches
        self.x = tf.placeholder(tf.float32,
                                shape=[self.batch_size, self.image_width, self.image_height, self.n_channels])
        self.y_ = tf.placeholder(tf.float32, shape=[self.batch_size, self.n_classes])

        # Convolutional layer #1 kernels/filters and bias
        self.kernel_conv_1 = self.weight_variable(self.kernel_conv + [self.n_channels] + [self.n_filters_1])
        self.bias_conv_1 = self.bias_variable([self.n_filters_1])

        # Convolutional layer #2 kernels/filters and bias
        self.kernel_conv_2 = self.weight_variable(self.kernel_conv + [self.n_filters_1] + [self.n_filters_2])
        self.bias_conv_2 = self.bias_variable([self.n_filters_2])

        # Dense layer #1 weights
        # W_dense_1 is dependent on the output size of the pool2 layer and is defined there
        self.W_dense_1 = None
        self.b_dense_1 = self.bias_variable((self.n_dense_1,))

        # Dense layer #2 weights
        self.W_dense_2 = self.weight_variable((self.n_dense_1, self.n_dense_2))
        self.b_dense_2 = self.bias_variable((self.n_dense_2,))

        # Dense layer #3 weights
        self.W_dense_3 = self.weight_variable((self.n_dense_2, self.n_classes))
        self.b_dense_3 = self.bias_variable((self.n_classes,))

    def run(self):

        # sess = tf.InteractiveSession()

        # Convolutional layer #1
        #conv1 = self.convolutional_layer(input_tensor=self.x, kernel=self.kernel_conv_1, bias=self.bias_conv_1,
         #                                stride=self.stride, padding=self.padding)
        
        conv1 = tf.layers.conv2d(
            inputs=self.x,
            filters=self.n_filters_1,
            kernel_size=[5, 5],
            strides = (2,2),
            padding="same",
            activation=tf.nn.relu)
        
        # Average pooling layer #1
        #pool1 = self.avg_pooling_layer(input_tensor=conv1, kernel=self.kernel_pool, padding=0,
         #                              stride=self.stride_pool)
    
        pool1 = tf.layers.average_pooling2d(inputs=conv1, pool_size=self.kernel_pool, strides=self.stride_pool)



        # Convolutional layer #2
       # conv2 = self.convolutional_layer(input_tensor=pool1, kernel=self.kernel_conv_2, bias=self.bias_conv_2,
        #                                 stride=self.stride, padding=0)
        
        conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=self.n_filters_2,
            kernel_size=[5, 5],
            padding="valid",
            activation=tf.nn.relu)

        # Average pooling layer #2
       # pool2 = self.avg_pooling_layer(input_tensor=conv2, kernel=self.kernel_pool, padding=0,
        #                               stride=self.stride_pool)
        
        pool1 = tf.layers.average_pooling2d(inputs=conv2, pool_size=self.kernel_pool, strides=self.stride_pool)

        
        # define size of dense_1 based on pool2 output
        pool2_size = pool2.get_shape().as_list()
        self.W_dense_1 = self.weight_variable([pool2_size[1] * pool2_size[2] * self.n_filters_2, self.n_dense_1])
        pool2 = tf.reshape(pool2, [-1, pool2_size[1] * pool2_size[2] * self.n_filters_2])
        
        dense1 = tf.layers.dense(inputs=pool2, units=self.n_dense_1, activation=tf.nn.relu)
        dense2 = tf.layers.dense(inputs=dense1, units=self.n_dense_2, activation=tf.nn.relu)
        logits = tf.layers.dense(inputs=dense2, units=self.n_classes, activation=tf.nn.relu)

        # Dense layer #1
       # dense1 = self.dense_layer(input_tensor=pool2, w=self.W_dense_1, b=self.b_dense_1)

        # Dense layer #2
       # dense2 = self.dense_layer(input_tensor=dense1, w=self.W_dense_2, b=self.b_dense_2)

        # Dense layer #3 / Logits layer
        #logits = self.dense_layer(input_tensor=dense2, w=self.W_dense_3, b=self.b_dense_3)

        # compute cross entroy for training
        cross_entropy = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.y_, logits=logits))
        optimizer = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(self.y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        with tf.Session() as sess:
            sess.run(tf.global_variables_initializer())
            for i in range(self.n_iter):
                batch = self.mnist.train.next_batch(self.batch_size)
                x = np.reshape(batch[0], (self.batch_size, 28, 28, 1))
                if i % 100 == 0:
                    train_accuracy = accuracy.eval(feed_dict={self.x: x, self.y_: batch[1]})
                    print('step %d, training accuracy %g' % (i, train_accuracy))
                    optimizer.run(feed_dict={self.x: x, self.y_: batch[1]})

            test_size = self.mnist.test.images.shape[0]
            acc = []
            for i in range(test_size // self.batch_size):
                batch = self.mnist.test.next_batch(self.batch_size)
                x = np.reshape(batch[0], (self.batch_size, 28, 28, 1))
                acc.append(accuracy.eval(feed_dict={self.x: x, self.y_: batch[1]}))

            print('test accuracy {}'.format(np.array(acc).mean()))

    def convolutional_layer(self, input_tensor, kernel, bias, stride, padding):
        # get input and filter dimensions
        n, height, width, channel = input_tensor.get_shape().as_list()
        filter_height, filter_width, filter_channel, filter_n = kernel.get_shape().as_list()

        # calculate output dimensions after convolution
        out_height = (height + 2 * padding - filter_height) // stride + 1
        out_width = (width + 2 * padding - filter_width) // stride + 1

        # flatten X and W for matrix multiplication
        x_flat = self.flatten(input_tensor, filter_height, filter_width, filter_channel, out_height, out_width, stride,
                              padding)
        w_flat = tf.reshape(kernel, [filter_height * filter_width * filter_channel, filter_n])

        # compute matrix multiplication
        z = tf.nn.xw_plus_b(x_flat, w_flat, bias)

        # reverse flatten and apply activation
        return self.activation(tf.transpose(tf.reshape(z, [out_height, out_width, n, filter_n]), [2, 0, 1, 3]))

    def avg_pooling_layer(self, input_tensor, kernel, padding, stride):
        # n, heigth, width, channel = [d.value for d in input_tensor.get_shape()]
        n, heigth, width, channel = input_tensor.get_shape()

        pool_height = kernel[0]
        pool_width = kernel[1]

        out_height = (heigth + 2 * padding - pool_height) // stride + 1
        out_width = (width + 2 * padding - pool_width) // stride + 1

        X_flat = self.flatten(input_tensor, pool_height, pool_width, channel, out_height, out_width, stride, padding)

        pool = tf.reduce_mean(tf.reshape(X_flat, [out_height, out_width, n, pool_height * pool_width, channel]), axis=3)
        return tf.transpose(pool, [2, 0, 1, 3])

    def dense_layer(self, input_tensor, w, b):
        return self.activation(tf.nn.xw_plus_b(input_tensor, w, b))

    @staticmethod
    def flatten(input_tensor, window_height, window_width, window_channel, out_height, out_width, stride,
                padding):

        # apply padding on image width and height and no padding on image and channel level
        X_padded = tf.pad(input_tensor, [[0, 0], [padding, padding], [padding, padding], [0, 0]])

        # iterate over height and width while taking stride into account
        # ignore image and channel dimensions
        windows = []
        for y in range(out_height):
            for x in range(out_width):
                windows.append(
                    tf.slice(X_padded, [0, y * stride, x * stride, 0],
                             [-1, window_height, window_width, -1]))
        # shape : [out_height * out_width, n, filter_height, filter_width, channel]
        stacked = tf.stack(windows)

        # return as 2D Tensor
        # second dimension contains value of one "stride"
        return tf.reshape(stacked, [-1, window_channel * window_width * window_height])

    @staticmethod
    def weight_variable(shape):
        return tf.Variable(tf.truncated_normal(shape=shape, stddev=0.1))

    @staticmethod
    def bias_variable(shape):
        return tf.Variable(tf.constant(.1, shape=shape))

In [38]:
LeNet().run()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


UnboundLocalError: local variable 'pool2' referenced before assignment